In [1]:
from neo4j.v1 import GraphDatabase, basic_auth 
driver = GraphDatabase.driver("bolt://localhost", auth=basic_auth("neo4j", "test"))

import json
import os
from datetime import datetime

photo_base_dir = '/Users/rvanweverwijk/Pictures/2016'

# watch("httpstream")


In [150]:
session = driver.session()
session.run("MERGE (d:Directory {name:'" + photo_base_dir + "'})")
session.close()

In [151]:
session = driver.session()
result = session.run("MATCH (d:Directory) RETURN d.name AS name")
for record in result:
    print(record["name"])
session.close()


/Users/rvanweverwijk/Pictures/2016


In [159]:
ignored_files = [".DS_Store"]

def get_current_time() :
    now = datetime.now()
    return int(now.strftime("%Y%m%d%H%M%S"))

def update_directory_structure(current_time):
    add_directories(current_time)
    update_modification_times()
    delete_non_existing_directories(current_time)
    add_photos(current_time)
    delete_non_existing_photos(current_time)

def create_directory_query(path, current_time):
    query = 'MERGE (root:Directory {name: "'+ photo_base_dir + '"})\nCREATE UNIQUE path = (root)'
    current_path = ''
    for dir in path:
        current_path = current_path + '/' + dir
        query = query + '-[:HAS_DIR]->(:Directory {name: "' + dir + '", fullPath: "' + current_path + '"})'
    query = query + '\nwith nodes(path) as dirs\nunwind dirs as dir\nset dir._lastModified = toInt(' + str(current_time) + ')'
    return query

def create_photo_query():
    query = '''
            MATCH (d:Directory {fullPath: {fullDirectoryPath}})\n
            MERGE (p:Photo {name:{fileName}, directory: {fullDirectoryPath}})\n
            set p._lastModified = {currentTime}\n
            MERGE (d)-[:CONTAINS_PHOTO]->(p)
    '''
    return query

def update_modification_times():
    session = driver.session()
    session.run('match (d:Directory) set d._timesModified = coalesce(d._timesModified, 0) + 1')
    session.close()
    
def add_directories(current_time):
    session = driver.session()
    with session.begin_transaction() as tx:
        for root, dirs, files in os.walk(photo_base_dir):
            if not dirs:
                path = root.replace(photo_base_dir + '/', '').split('/')
                tx.run(create_directory_query(path, current_time))
        tx.success = True
    session.close()
        
def add_photos(current_time):
    print("add photos")
    session = driver.session()
    with session.begin_transaction() as tx:
        for root, dirs, files in os.walk(photo_base_dir):
            path = root.replace(photo_base_dir, '')
            for file in files: 
                if (file not in ignored_files):
                    result = tx.run(statement=create_photo_query(),parameters={"currentTime": current_time, "fullDirectoryPath": path, "fileName": file})
        tx.success = True
    session.close()

def delete_non_existing_directories(current_time) :
    print("delete directories")
    session = driver.session()
    with session.begin_transaction() as tx:
        result = tx.run(statement='match (d:Directory) where d._lastModified < {currentTime} detach delete d', parameters={"currentTime": current_time})
        sum = result.consume().counters
        print(sum)
        tx.success = True
    session.close

def delete_non_existing_photos(current_time) :
    print("delete photos")
    session = driver.session()
    with session.begin_transaction() as tx:
        result = tx.run(statement='match (p:Photo) where p._lastModified < {currentTime} detach delete p', parameters={"currentTime": current_time})
        sum = result.consume().counters
        print(sum)
        tx.success = True
    session.close

In [161]:
current_time = get_current_time()
update_directory_structure(current_time)


delete directories
{}
add photos
delete photos
{}


In [89]:
current_time

20160930123232